Задание предлагается сделать, используя язык Python3
	
На входе есть 
таблица отделений банка (кол-во записей 100 – 10 000) с 
полями DIV_ID (ID отделения), CUR_CAPACITY (целое – текущее кол-во клиентов в отделении), 
MAX_CAPACITY (целое – максимальное кол-во клиентов в отделении)

И таблица с клиентами (кол-во записей 1000 – 1 000 000), которым нужно рекомендовать отделение для посещения,
                                   с полями CLIENT_ID, DIV1_ID (самое предпочтительное отделение для клиента), 
                                                       DIV2_ID  (второе предпочтительное отделение для клиента),
                                                       DIV3_ID  (третье предпочтительное отделение для клиента)
                 
Результатом работы программы должна быть таблица CLIENT_ID, DIV_ID
в которой мы для каждого клиента рекомендуем одно из отделений DIV1_ID, DIV2_ID, DIV3_ID
При этом новая CUR_CAPACITY для каждого отделения (считаем, что каждый клиент 
согласился  на рекомендацию) не должна превышать MAX_CAPACITY.     
(в качестве входных и выходных таблиц можно использовать Pandas dataframe или
иную подходящую структуру данных)

### Подготовка среды

In [1]:
import numpy as np
import pandas as pd

### Считывание входных данных, их обработка и создание DataFrame для итогового распределения

In [2]:
# Загрузка данных клиентов, установка ID клиента в качестве индекса
clients = pd.read_excel("./KLI.xlsx").set_index('CLIENT_ID') 

# Загрузка данных отделений, установка ID отделения в качестве индекса
banks = pd.read_excel("./BANK.xlsx").set_index('DIV_ID')

# Создание вспомогательного столбца FREE_CAPACITY, который соответствует количеству свободных мест в отделении
banks['FREE_CAPACITY'] = banks['MAX_CAPACITY'] - banks['CUR_CAPACITY']

# Создание DataFrame для итогового распределния клиентов по отделениям
# В качестве индекса устонавливается ID клиента
distribution = pd.DataFrame(columns=['CLIENT_ID', 'DIV_ID'], dtype='int64').set_index('CLIENT_ID')

### Распределение клиентов по их приоритетным отделениям

In [3]:
# Алгоритм заключается в следующем:
# Каждый клиент распределяется в свое первое по приоритету отделение
# Отбираются те клиенты, которым не хватило места в своем приоритетном отделении
# Далее эти клиенты распределяются в свое второе по приоритету отеделение
# Если и в этот раз для некоторых из клиентов не хватило места в этих отделениях
# Они распределяются в свое третье по приоритету отеделение 
for div in ['DIV1_ID', 'DIV2_ID', 'DIV3_ID']:
    
    # grouped_clients - это клиенты сгруппированные по ID текущих приоритетных направлений (DIV1_ID, DIV2_ID, DIV3_ID),
    # которым присвоены номера в своей группе 
    grouped_clients = clients.groupby(div).cumcount().to_frame().rename(columns={0: 'NUMBER_IN_DIV'})
    
    # div_capacities - количество свободных мест в каждом из отделений (используется для маски)
    div_capacities = banks.loc[clients.loc[grouped_clients.index.values][div]]['FREE_CAPACITY'].values
    
    # numbers_in_divs - это порядковые номера клиентов в своем приоритетном отделении (используется для маски)
    numbers_in_divs = grouped_clients['NUMBER_IN_DIV'].values
    
    # Отбор тех клиентов, для которых нашлись свободные места в отделениях
    grouped_clients = grouped_clients[numbers_in_divs < div_capacities]
    
    # Создается новый столбец, который предстовляет собой ID отделения, в которое попал клиент
    grouped_clients = pd.merge(grouped_clients, clients.rename(columns={div: 'DIV_ID'})[
        ['DIV_ID']], left_index=True, right_index=True)
    
    # Учёт изменения количества свободных мест в отделениях:
    # Для изменения количества свободных мест в отделениях создается пустой DataFrame с индексами отделений, заполненный нулями
    # Из этого DataFrame вычитается количество клиентов попавших в каждое из отделений
    # Так как на этом этапе могут возникнуть NaN значения (не все отделения могли быть задействованы)
    # Эти значения заменяются нулями
    # Получившийся DataFrame представляет собой изменение количества свободных мест в каждом из отделений, 
    # которые в последствии складываются со столбцом FREE_CAPACITY 
    banks['FREE_CAPACITY'] += (pd.DataFrame(data=np.zeros(shape=banks.index.size),
                                            dtype='int64',
                                            index=banks.index)
                               .iloc[:, 0] - grouped_clients.groupby('DIV_ID').count().iloc[:, 0]).fillna(0)
    
    # Сохранение информации о клиентах, которые были распределены 
    distribution = pd.concat([distribution['DIV_ID'], grouped_clients['DIV_ID']]).to_frame()
    
    # Удаление информации о распределённых клиентах и переход к следующим по приоритету отделениям 
    # (для нераспределённых клиентов)
    clients.drop(grouped_clients.index, inplace=True)

### Вывод результатов

#### Нераспределённые клиенты

In [4]:
clients #Клиенты, которые никуда не прошли

,DIV1_ID,DIV2_ID,DIV3_ID
CLIENT_ID,,,
1000,98,98,98


#### Распределённые клиенты

In [5]:
distribution.sort_index() 

,DIV_ID
CLIENT_ID,
1,10
2,65
3,1
4,95
5,71
...,...
995,14
996,27
997,76
